# RNN embedding
This notebook focus on the application of LSTM to embed the sentence and measure the similarity between question pairs
<br>
Reference: ***Siamese Recurrent Architectures for Learning Sentence Similarity***
<br>http://www.mit.edu/~jonasm/info/MuellerThyagarajan_AAAI16.pdf

In [1]:
import pandas as pd
import numpy as np
from gensim import corpora, models, similarities
import re
import pickle
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow import nn
from sys import getsizeof

In [2]:
w2v_model = pickle.load(open("w2v_model.dat", "rb"))

# Load and preprocess

In [3]:
# # clean the sentence
# def txt_clean(sentence):
#     s = re.sub("[^a-zA-Z0-9]", " ", str(sentence))
#     s_list = s.lower().split()
#     return s_list

# train = pd.read_csv("../data/train.csv")
# test = pd.read_csv("../data/test.csv")

# # transform to get split sentence
# train_q1 = train.apply(lambda x: txt_clean(x["question1"]),axis = 1).tolist()
# train_q2 = train.apply(lambda x: txt_clean(x["question2"]),axis = 1).tolist()
# test_q1 = test.apply(lambda x: txt_clean(x["question1"]),axis = 1).tolist()
# test_q2 = test.apply(lambda x: txt_clean(x["question2"]),axis = 1).tolist()
# y = train.is_duplicate.tolist()

# pickle.dump(train_q1, open("../data/train_q1.dat", "wb"))
# pickle.dump(train_q2, open("../data/train_q2.dat", "wb"))
# pickle.dump(test_q1, open("../data/test_q1.dat", "wb"))
# pickle.dump(test_q2, open("../data/test_q2.dat", "wb"))
# pickle.dump(y, open("../data/y.dat", "wb"))

In [4]:
train_q1 = pickle.load(open("../data/train_q1.dat", "rb"))
train_q2 = pickle.load(open("../data/train_q2.dat", "rb"))
# test_q1 = pickle.load(open("../data/test_q1.dat", "rb"))[0:1000]
# test_q1 = pickle.load(open("../data/test_q1.dat", "rb"))[0:1000]

In [5]:
target = pickle.load(open("../data/y.dat", "rb"))

target = np.array(target).reshape(-1,1)

target_inverse = 1-target

target = np.concatenate([target, target_inverse], 1)

# Build a LSTM for embedding

1.Test the word2vec model

In [6]:
# trainsform to word vectors
for w in train_q1[1]:
    print(w2v_model.wv[w][0:5])

[ 0.13234842  0.00763958 -0.08464085 -0.08113342  0.24815077]
[ 0.10499778  0.06805725 -0.10598193 -0.03042802  0.03230476]
[-0.0287225  -0.07091928  0.00331686 -0.04636408 -0.02650626]
[-0.51597834 -0.08152942 -0.45761555  0.61214203 -0.25413761]
[-0.00364251  0.19204271 -0.17447464 -0.03469274 -0.12318879]
[ 0.04679416 -0.11005198 -0.65378165  0.05505853  0.09023309]
[-0.22974356  0.58205104 -0.12804945 -0.42072222 -0.47509399]
[ 0.13577995 -0.19249639 -0.3507593   0.00056107  0.06072884]
[-0.41628373  0.11394925 -0.59150016 -0.15818399  0.12219065]
[-0.03838921  0.43435898 -0.22790641 -0.10768311  0.15719041]


The model is OK to transform the word to vectors

2.padding the list to the same length

In [16]:
len(train_q1)

5000

In [17]:
train_q1[0]

['what',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india']

In [18]:
train_q1[2]

['how',
 'can',
 'i',
 'increase',
 'the',
 'speed',
 'of',
 'my',
 'internet',
 'connection',
 'while',
 'using',
 'a',
 'vpn']

In [19]:
testdata = train_q1[0:10]

In [22]:
n_input = 300

In [24]:
# for loop to padd
maxLen = getMaxLength(testdata)

testdatapad = padding(testdata, maxLen)

transdata = transToVec(testdatapad,maxLen)

In [25]:
transdata.shape

(10, 17, 300)

Using traditional RNN cell first

1.Model graph

In [6]:
# functions for padding list
def getMaxLength(sentences):
    "setences: list of list of words"
    maxLen = -1
    for s in sentences:
        if len(s) > maxLen:
            maxLen = len(s)
    return maxLen

def padding(sentences, maxLen):
    padContent = "thisispadding"
    sentencesPad = []
    for s in sentences:
        s.extend([padContent]*(maxLen-len(s)))
        sentencesPad.append(s)
    return sentencesPad

def transToVec(sentences,maxLen):
    transSentence = []
    for s in sentences:
        sen = []
        for j in range(maxLen):
            if s[j] == "thisispadding":
                sen.append([0] * n_input)
            else:
                sen.append(w2v_model.wv[s[j]].tolist())
        transSentence.append(sen)
    return np.array(transSentence) 

In [7]:
# reset all parameters
tf.reset_default_graph()

In [8]:
# model parameters
n_hidden = 64
n_input = 300
n_classes = 2
n_dense1 = 256
# n_dense2 = 256

learning_rate = 0.01
lambda_loss_amount = 0.001
batch_size = 512
training_iters = 20*batch_size
display_iter = 100

In [9]:
# Graph input
x1 = tf.placeholder(tf.float64, [None, None, n_input])
x2 = tf.placeholder(tf.float64, [None, None, n_input])
y_ = tf.placeholder(tf.float64, [None, n_classes])

# dense layer variable
W1 = tf.Variable(tf.random_normal([2*n_hidden,n_classes],dtype=tf.float64))
b1 = tf.Variable(tf.random_normal([n_classes],dtype=tf.float64))

# W2 = tf.Variable(tf.random_normal([n_dense1,n_classes],dtype=tf.float64))
# b2 = tf.Variable(tf.random_normal([n_classes],dtype=tf.float64))

# W3 = tf.Variable(tf.random_normal([n_dense2,n_classes],dtype=tf.float64))
# b3 = tf.Variable(tf.random_normal([n_classes],dtype=tf.float64))

In [10]:
# RNN embedding
def RNN(x1, x2, W1, b1):
    
    with tf.variable_scope('embedding_1'):
        rnn_cell_1 = rnn.BasicLSTMCell(n_hidden)
        outputs_1, states_1 = nn.dynamic_rnn(rnn_cell_1, x1, dtype=tf.float64)
        
    with tf.variable_scope('embedding_2'):
        rnn_cell_2 = rnn.BasicLSTMCell(n_hidden)
        outputs_2, states_2 = nn.dynamic_rnn(rnn_cell_2, x2, dtype=tf.float64)

    # concat two embedding into one
    sentence_embedding = tf.concat([states_1[0], states_2[0]], axis=1)
    
    with tf.variable_scope("dense_layer"):
        y = tf.matmul(sentence_embedding,W1) + b1
#     a1 = tf.nn.relu(tf.matmul(sentence_embedding,W1) + b1)
#     a2 = tf.matmul(a1,W2) + b2
#     a3 = tf.matmul(a2,W3) + b3

    return y

In [11]:
# prediction
y = RNN(x1, x2, W1, b1)

# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# SGD
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [12]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float64))

In [13]:
# batch generating
N = target.shape[0]
batch_index = np.arange(0,N)
batch_start = np.append(np.arange(0,N,batch_size),N)

model test with 5000 training queries

In [1]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    step = 1
    while step < 100:
        
        # shuffle data
        np.random.shuffle(batch_index)
        
        for i in range(len(batch_start)-1):
            # Batch processing 
            batch_x_1= train_q1[batch_start[i]:batch_start[i+1]]
            maxLen_1 = getMaxLength(batch_x_1)
            batch_x_pad_1 = padding(batch_x_1, maxLen_1)
            batch_x_trans_1 = transToVec(batch_x_pad_1,maxLen_1)
            batch_x_2= train_q2[batch_start[i]:batch_start[i+1]]
            maxLen_2 = getMaxLength(batch_x_2)
            batch_x_pad_2 = padding(batch_x_2, maxLen_2)
            batch_x_trans_2 = transToVec(batch_x_pad_2,maxLen_2)
            batch_y = target[batch_start[i]:batch_start[i+1],:]

            # train a hidden layer
            sess.run(train_step, feed_dict={x1:batch_x_trans_1,
                                            x2:batch_x_trans_2,
                                            y_: batch_y})

        # print loss and accuracy
        print("Train Cross-Entropy Loss", sess.run(cross_entropy, feed_dict={x1:batch_x_trans_1,
                                            x2:batch_x_trans_2,
                                            y_: batch_y}))
        
        print("Train Accuracy:", sess.run(accuracy, feed_dict={x1:batch_x_trans_1,
                                                                x2:batch_x_trans_2,
                                                                y_: batch_y}))
        # update batch
        step += 1